<h1>DOWNTUBE</h1>

In [1]:
import inquirer
from pyfiglet import Figlet
from pytube.__main__ import YouTube
from rich import print
from pytube import Stream
import time
import sys

In [2]:
figlet = Figlet(font='big')
print(f"[red]{figlet.renderText('Downtube')}[/red]")

 _____                      _         _          
|  __ \                    | |       | |         
| |  | | _____      ___ __ | |_ _   _| |__   ___ 
| |  | |/ _ \ \ /\ / / '_ \| __| | | | '_ \ / _ \
| |__| | (_) \ V  V /| | | | |_| |_| | |_) |  __/
|_____/ \___/ \_/\_/ |_| |_|\__|\__,_|_.__/ \___|
                                                 
                                                 

In [3]:
video_url = "https://youtu.be/MwtKJG_87fw?si=ysG17gJK52FsFny9"
url = 'https://youtu.be/gh3FyLT7WVg?list=RDgh3FyLT7WVg'
url_supa = 'https://youtu.be/dU7GwCOgvNY?si=ovCJGbiQKdaFoVXm'

In [4]:
yt = YouTube(url=url)
video = yt.streams.filter(only_video=True)
audio = yt.streams.filter(only_audio=True)

In [5]:
resolution = dict()
for x in video:
        resolution[x.itag] = [x.resolution,x.fps,x.filesize_mb,x.mime_type]

In [6]:
resolution

{313: ['2160p', 24, 526.128, 'video/webm'],
 271: ['1440p', 24, 250.595, 'video/webm'],
 137: ['1080p', 24, 121.574, 'video/mp4'],
 248: ['1080p', 24, 60.995, 'video/webm'],
 136: ['720p', 24, 40.751, 'video/mp4'],
 247: ['720p', 24, 36.981, 'video/webm'],
 135: ['480p', 24, 27.761, 'video/mp4'],
 244: ['480p', 24, 18.994, 'video/webm'],
 134: ['360p', 24, 16.738, 'video/mp4'],
 243: ['360p', 24, 12.277, 'video/webm'],
 133: ['240p', 24, 7.345, 'video/mp4'],
 242: ['240p', 24, 4.92, 'video/webm'],
 160: ['144p', 24, 3.363, 'video/mp4'],
 278: ['144p', 24, 2.482, 'video/webm']}

In [7]:
audio

[<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">, <Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="250" mime_type="audio/webm" abr="70kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">]

In [8]:
resolution = dict()
for x in audio:
        resolution[x.itag] = [x.mime_type,x.abr,x.filesize_mb]

In [9]:
resolution

{139: ['audio/mp4', '48kbps', 1.645],
 140: ['audio/mp4', '128kbps', 4.362],
 249: ['audio/webm', '50kbps', 1.711],
 250: ['audio/webm', '70kbps', 2.226],
 251: ['audio/webm', '160kbps', 4.339]}

In [10]:
highest_quality_stream = None
highest_bitrate = 0

    # print(audio)

    # # Iterate through the streams to find the highest quality audio
for stream in audio:
    if stream.type == "audio":
        bitrate = int(stream.abr.replace("kbps", ""))
        if bitrate > highest_bitrate:
            highest_bitrate = bitrate
            highest_quality_stream = stream

In [11]:
highest_quality_stream

<Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">

In [12]:
def on_progress(
    stream: Stream, chunk: bytes, bytes_remaining: int
) -> None:  # pylint: disable=W0613
    filesize = stream.filesize
    bytes_received = filesize - bytes_remaining
    display_progress_bar(bytes_received, filesize)

start_time = time.time()

In [13]:

def display_progress_bar(
    bytes_received: int, filesize: int, ch: str = "▬", scale: float = 0.55
) -> None:
    """Display a simple, pretty progress bar.

    Example:
    ~~~~~~~~
    PSY - GANGNAM STYLE(강남스타일) MV.mp4
    ↳ |▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬| 100.0%

    :param int bytes_received:
        The delta between the total file size (bytes) and bytes already
        written to disk.
    :param int filesize:
        File size of the media stream in bytes.
    :param str ch:
        Character to use for presenting progress segment.
    :param float scale:
        Scale multiplier to reduce progress bar size.

    """
    columns = shutil.get_terminal_size().columns
    max_width = int(columns * scale)

    filled = int(round(max_width * bytes_received / float(filesize)))
    remaining = max_width - filled
    progress_bar = ch * filled + " " * remaining
    # 
    downloaded_size = f"{bytes_received / (1024 * 1024):.2f}MB"
    total_size = f"{filesize / (1024 * 1024):.2f}MB"
    elapsed_time = time.time() - start_time
    download_speed = bytes_received / elapsed_time  # Bytes per second
    time_left = (filesize - bytes_received) / download_speed
    time_left_str = time.strftime("%H:%M:%S", time.gmtime(time_left))
    # 
    percent = round(100.0 * bytes_received / float(filesize), 1)
    text = f"↳|{progress_bar}| {percent}% - {downloaded_size}/{total_size} - {time_left_str} left\r"
    sys.stdout.write(text)
    sys.stdout.flush()


In [15]:
import tempfile
import os
video_file = tempfile.TemporaryFile(
    dir='./',
    prefix="vid",
    suffix='.webm',
    delete=False
)
print('File created in the cwd')
print(video_file.name)

file_path = video_file.name
# Get the directory (folder) of the file
directory = os.path.dirname(file_path)

# Get the filename (with extension) from the file path
filename = os.path.basename(file_path)

print("Directory:", directory)
print("Filename:", filename)

yt.streams.get_by_itag(248).download(output_path=directory,filename=filename)

File created in the cwd

d:\TryOn\DownTube\vid5djr1txl.webm

Directory: d:\TryOn\DownTube

Filename: vid5djr1txl.webm

'd:\\TryOn\\DownTube\\vid5djr1txl.webm'

In [ ]:
# video_file.close()

In [16]:
audio_file = tempfile.TemporaryFile(
    dir='./',
    prefix="vid",
    suffix='.webm',
    delete=False
)
print('File created in the cwd')
print(audio_file.name)

file_path = audio_file.name
# Get the directory (folder) of the file
directory = os.path.dirname(file_path)

# Get the filename (with extension) from the file path
a_filename = os.path.basename(file_path)

print("Directory:", directory)
print("Filename:", a_filename)

yt.streams.get_by_itag(251).download(output_path=directory,filename=filename)

File created in the cwd

d:\TryOn\DownTube\vidxahre71k.webm

Directory: d:\TryOn\DownTube

Filename: vidxahre71k.webm

'd:\\TryOn\\DownTube\\vid5djr1txl.webm'

In [ ]:
# os.remove(filename)

In [17]:
from moviepy.editor import VideoFileClip, AudioFileClip
import traceback

def combine_video_audio(video_path, audio_path, output_path):
    try:
        # Load the video and audio files
        video_clip = VideoFileClip(video_path)
        audio_clip = AudioFileClip(audio_path)

        # Get metadata of video file
        video_fps = video_clip.fps
        video_size = video_clip.size
        video_rotation = video_clip.rotation

        # Set the audio of the video clip to the loaded audio clip
        video_clip = video_clip.set_audio(audio_clip)

        # Write the combined video with audio to a new file
        video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

        # Close the clips
        video_clip.close()
        audio_clip.close()

    except KeyError as e:
        # Handle KeyError
        print(f"Error: {e}")
        print("Video metadata is missing required keys.")
        traceback.print_exc()  # Print traceback for debugging

if __name__ == "__main__":
    video_path = filename
    audio_path = a_filename
    output_path = "output_combined_moviepy.mp4"

    combine_video_audio(video_path, audio_path, output_path)


Error: 'video_fps'

Video metadata is missing required keys.

Traceback (most recent call last):
  File "C:\Users\NEEVEN PRADHAPH\AppData\Local\Temp\ipykernel_19140\2277148748.py", line 7, in combine_video_audio
    video_clip = VideoFileClip(video_path)
  File "d:\TryOn\DownTube\.venv\lib\site-packages\moviepy\video\io\VideoFileClip.py", line 88, in __init__
    self.reader = FFMPEG_VideoReader(filename, pix_fmt=pix_fmt,
  File "d:\TryOn\DownTube\.venv\lib\site-packages\moviepy\video\io\ffmpeg_reader.py", line 37, in __init__
    self.fps = infos['video_fps']
KeyError: 'video_fps'


In [ ]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np

def combine_video_audio(video_path, audio_path, output_path):
    # Read video
    video_capture = cv2.VideoCapture(video_path)
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    total_frames = int(video_capture.get(cv.CAP_PROP_FRAME_COUNT))

    # Read audio
    audio_capture = cv2.VideoCapture(audio_path)
    sample_rate = int(audio_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    num_channels = int(audio_capture.get(cv2.CAP_PROP_CHANNEL_COUNT))

    # FourCC code for video codec
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Create video writer object
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Loop through each frame in the video
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Write frame to video writer
        video_writer.write(frame)

    # Release video capture and writer
    video_capture.release()
    video_writer.release()

    # Combine audio and video using ffmpeg
    cmd = f"ffmpeg -i {output_path} -i {audio_path} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 -y final_video.mp4"
    subprocess.call(cmd, shell=True)

if __name__ == "__main__":
    video_path = "vid99d5nto3.webm"
    audio_path = "vidjcfxfhku.webm"
    output_path = "output_combined.mp4"

    combine_video_audio(video_path, audio_path, output_path)


NameError: name 'cv' is not defined

In [18]:
import av

def combine_video_audio(video_path, audio_path, output_path):
    # Open video and audio files
    video_container = av.open(video_path)
    audio_container = av.open(audio_path)

    # Create output container
    output_container = av.open(output_path, 'w')

    # Add video stream to output container
    video_stream = output_container.add_stream(template=video_container.streams.video[0])

    # Add audio stream to output container
    audio_stream = output_container.add_stream(template=audio_container.streams.audio[0])

    # Mux video frames
    for frame in video_container.decode(video=0):
        packet = video_stream.encode(frame)
        output_container.mux(packet)

    # Mux audio frames
    for frame in audio_container.decode(audio=0):
        packet = audio_stream.encode(frame)
        output_container.mux(packet)

    # Close containers
    output_container.close()
    video_container.close()
    audio_container.close()

if __name__ == "__main__":
    video_path = filename
    audio_path = a_filename
    output_path = "output_combined.mp4"

    combine_video_audio(video_path, audio_path, output_path)


Format matroska,webm detected only with low score of 1, misdetection possible!
EBML header parsing failed


InvalidDataError: [Errno 1094995529] Invalid data found when processing input: 'vidxahre71k.webm'; last error log: [matroska,webm] EBML header parsing failed